<a href="https://colab.research.google.com/github/T2718/AI_Make_Image/blob/main/AI6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diffusers transformers accelerate safetensors

import os
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
from PIL import Image
from safetensors.torch import load_file
import torch


model_list = ['stablediffusionapi/eleet-model','stablediffusionapi/brav6','Vsukiyaki/ShiratakiMix']
model_id = ''
while True:
  set = input('設定(未入力でOK)')
  if set == 'model':
    model_id = input('model_idを入力')
  elif set == 'model_list':
    for k in range(len(model_list)):
      print(str(1+k)+':'+model_list[k])
    model_id = model_list[int(input('何番を使用しますか？'))-1]

  elif set != '':
    print('設定が見つかりません。')
    continue
  else:
    break



if model_id == '':
  # Stable Diffusionのモデルをロード
  model_id = "stablediffusionapi/eleet-model"


# ユーザーからの入力を取得
prompt = input("プロンプトを入力してください (強さをつける場合は形式: 'prompt1:weight, prompt2:weight'): ")
negative_prompt = input("ネガティブプロンプトを入力してください: ")
num_images = int(input("生成する画像の枚数を入力してください: "))
num_inference_steps = int(input("推論ステップ数を入力してください: "))
mode = input("text to image または image to image を選んでください (t/i): ")

HugCiv = input('Hugging Face(H) or CivitAI(C): ')
if HugCiv == 'C':
  model_id = "./models/Stable-diffusion/convert_model"

!mkdir output

# フォルダ名をプロンプトの前15文字にする
output_dir = 'output/'+prompt[:15].replace(" ", "_")  # スペースをアンダースコアに置き換え
os.makedirs(output_dir, exist_ok=True)

# 画像生成
if mode.lower() == 't':
    pipe = StableDiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        safety_checker=None  # Safe checkを無効に
    )
    pipe = pipe.to("cuda")  # GPUが必要です
    k_count = 0
    original_output = output_dir
    while True:
        output_dir = original_output+'/'+str(k_count)
        os.makedirs(output_dir, exist_ok=True)
        for i in range(num_images):
            image = pipe(
                prompt=prompt,
                negative_prompt=negative_prompt,
                num_inference_steps=num_inference_steps
            ).images[0]
            # 画像をファイルに保存 (数字のファイル名)
            image.save(os.path.join(output_dir, f"{i + 1}.png"))
        k_plus = input('追加数(しない場合は0): ')
        if(k_plus == '0'):
            break
        print(prompt)
        k_prompt = input('プロンプト(変更しない場合は無入力): ')
        print(negative_prompt)
        k_negative_prompt = input('ネガティブプロンプト(変更しない場合は無入力): ')
        if k_prompt != '':
            prompt = k_prompt
        if k_negative_prompt != '':
            negative_prompt = k_negative_prompt

        num_images = int(k_plus)
        k_count += 1

elif mode.lower() == 'i':
    image_path = input("元画像のpathを入力してください(例: img.png): ")
    strength = float(input("元画像の変更量(0~1): "))
    original_image = Image.open(image_path).convert("RGB")

    # Image-to-Imageパイプライン
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        safety_checker=None
    )
    pipe = pipe.to("cuda")


    for i in range(num_images):
        image = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            image=original_image,
            strength=strength,  # 元画像をどれだけ変化させるか (0～1)
            num_inference_steps=num_inference_steps
        ).images[0]
        # 画像をファイルに保存 (数字のファイル名)
        image.save(os.path.join(output_dir, f"{i + 1}.png"))

print(f"生成された画像は '{output_dir}' フォルダに保存されました。")


設定(未入力でOK)
プロンプトを入力してください (強さをつける場合は形式: 'prompt1:weight, prompt2:weight'): Cat ears, anime girl, lineart, monochrome, only black, edge only
ネガティブプロンプトを入力してください: color, real
生成する画像の枚数を入力してください: 10
推論ステップ数を入力してください: 120
text to image または image to image を選んでください (t/i): i
Hugging Face(H) or CivitAI(C): 
mkdir: cannot create directory ‘output’: File exists
元画像のpathを入力してください(例: img.png): IMG_0044.jpeg
元画像の変更量(0~1): 1


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_img2img.StableDiffusionImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

In [ ]:
!pip install transformers

from transformers import CLIPTokenizer

tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")

prompt = input('prompt : ')
tokens = tokenizer(prompt)["input_ids"]

print("トークン数:", len(tokens))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-bd3f826e9d99>", line 7, in <cell line: 0>
    prompt = input('prompt : ')
             ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 1177, in raw_input
    return self._input_request(
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 1219, in _input_request
    raise KeyboardInterrupt("Interrupted by user") from None
KeyboardInterrupt: Interrupted by user

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
          ^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'Keyb

TypeError: object of type 'NoneType' has no len()

In [ ]:
!rm -r output